In [1]:
# 0. Imports e configuração de paths
import pandas as pd, numpy as np, umap
from sklearn.cluster import KMeans
from pathlib import Path, PurePath
import matplotlib.pyplot as plt, seaborn as sns, json, os

In [2]:
# Ajuste o caminho raiz do repositório se necessário
ROOT = Path().resolve()  # assume que o notebook vive na raiz ou em notebooks/
DATA = ROOT / 'data'
DATA.mkdir(exist_ok=True)

In [3]:
EMB_CSV = DATA / 'symbolic_metrics_embeddings.csv'
assert EMB_CSV.exists(), f'Arquivo não encontrado: {EMB_CSV}'

AssertionError: Arquivo não encontrado: /Users/demetriosagourakis/Library/Mobile Documents/com~apple~CloudDocs/Biologia Fractal/entropic-symbolic-society/NHB_Symbolic_Mainfold/notebook/data/symbolic_metrics_embeddings.csv

In [ ]:
df = pd.read_csv(EMB_CSV)
print('Loaded', df.shape, 'rows')

In [ ]:
# 1. Seleciona coluna(s) de embeddings
embedding_cols = [c for c in df.columns if c.startswith('emb_') or c.startswith('dim_')]
assert len(embedding_cols) > 2, 'Não foram encontradas colunas de embedding suficientes.'
X = df[embedding_cols].values.astype('float32')
print('Embedding matrix shape:', X.shape)

In [ ]:
reducer = umap.UMAP(n_neighbors=30, min_dist=0.1, metric='cosine', random_state=42)
X_umap = reducer.fit_transform(X)
print('UMAP shape:', X_umap.shape)

In [ ]:
# Salva array
np.save(DATA / 'X_umap.npy', X_umap)
print('Saved X_umap.npy')

In [ ]:
n_clusters = 8  # ajuste conforme análise de silhueta ou conforme notebook 05
kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init='auto')
labels = kmeans.fit_predict(X_umap)
df['label'] = labels
np.savetxt(DATA / 'labels.csv', labels, delimiter=',', fmt='%d')
print('Saved labels.csv')

In [ ]:
df[['x_umap', 'y_umap']] = X_umap
consolidated_path = DATA / 'symbolic_umap_dataframe.csv'
df.to_csv(consolidated_path, index=False)
print('Consolidated DataFrame saved to', consolidated_path)

In [ ]:
sns.set(style='white', rc={'figure.figsize':(8,6)})
sns.scatterplot(x='x_umap', y='y_umap', hue='label', data=df, palette='tab10', s=25, linewidth=0)
plt.title('UMAP projection with K‑Means labels')
plt.axis('equal')
plt.show()

In [ ]:
params = {
    'umap': {'n_neighbors': 30, 'min_dist': 0.1, 'metric': 'cosine'},
    'kmeans': {'n_clusters': int(n_clusters), 'random_state': 42}
}
with open(DATA / 'umap_kmeans_params.json', 'w') as f:
    json.dump(params, f, indent=2)
print('Saved umap_kmeans_params.json')